**This notebook experiments with joining the datasets I created. 
After this notebook, I stopped experimenting and started working towards a definitive business question and model to support the business case. This aided me in understanding what to join and drop from the data I had collected.**

**You can see notes in this notebook that show I was still coming up with different ideas within this dataset. I have kept this for future iterations.**

#### Additional Data Options if interested in more (need a good reason, because you have  A LOT)
- Continue down artist path - join with kaggle dataset
    - https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data_by_artist_o.csv
    - data_by_artist_o.csv
    - but this is by artist name... 
- Go down genre path
    - https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data_by_artist_o.csv
    - data_by_genres_o.csv

In [1]:
# Basics 
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

# Text Cleaning
import re

#Spotify
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time 

# Importing dfs with heavy processing
import pickle

In [2]:
# import peloton data
# Thank you to okaykristinakay on Reddit for the data!
# Source: https://www.reddit.com/r/pelotoncycle/comments/m18xnr/peloton_class_list_march_update/
df = pd.read_excel('../../../data/original_datasets/AGF_Peloton Classes March.xlsx')
df.head()

,classId,className,classDescription,classDifficulty,classDuration,classType,classLength,classLocation,classOriginalAirdate,classRating,classRatingCount,instructorName,instructorBio,classEquipment,classSongs,classArtists,classUrl
0,7f66378211c9476b9b5619bf989f91d0,20 min Peace Meditation,A guided meditation that focuses on cultivatin...,4.3333,20,Meditation,23,psny-studio-2,2021-09-03 13:25:00,0.9847,131,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...",Yoga Block,Meditation 22,RIOPY,https://members.onepeloton.com/classes/bootcam...
1,54ac61803b364b2fa8378acd9f593cdb,15 min Bodyweight Strength,"No equipment, no problem. Join us for a high-e...",5.7755,15,Strength,19,psny-studio-3,2021-09-03 13:19:00,0.9933,297,Olivia Amato,"Born and raised in New York, Olivia grew up pl...",Workout Mat,"California Gurls,Let's Get Loud,Let It Rock (f...","Katy Perry,Snoop Dogg,Jennifer Lopez,Kevin Rud...",https://members.onepeloton.com/classes/bootcam...
2,c75fd4831573483c9d45739aae11d083,20 min Focus Flow: Lower Body,This yoga flow class focuses on poses that eng...,4.3664,20,Yoga,23,psny-studio-2,2021-09-03 12:51:00,1.0000,159,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...","Yoga Blanket,Yoga Block,Yoga Mat","Interlude No 1,Oceansize,She Just Likes To Fig...","James Vincent McMorrow,Oh Wonder,Four Tet,Grim...",https://members.onepeloton.com/classes/bootcam...
3,470086936f7a4723ab5a53cb80b571ff,45 min Pop Bootcamp,Split your workout 50/50 between cardio on the...,7.8312,45,Tread Bootcamp,50,psny-studio-4,2021-09-03 11:56:00,0.9737,152,Olivia Amato,"Born and raised in New York, Olivia grew up pl...","Workout Mat,Medium Weights",34+35 (Remix) (feat. Doja Cat & Megan Thee Sta...,"Ariana Grande,Doja Cat,Megan Thee Stallion,Jus...",https://members.onepeloton.com/classes/bootcam...
4,9680a817bf2149d2b91990c87166a400,20 min Pop Ride,We dare you not to dance as you ride to all th...,7.4000,20,Cycling,24,uk,2021-09-03 07:52:00,1.0000,82,Sam Yo,Sam is a pro at many things but shines when it...,None,"Señorita,Marry You,Irreplaceable,What Do You M...","Justin Timberlake,Bruno Mars,Beyoncé,Justin Bi...",https://members.onepeloton.com/classes/bootcam...


In [3]:
# Opening pickle files
# Use classId where you can as key for merging
base_clean_df = pd.read_pickle("../../../data/pickled_dfs/base_clean_df.pkl")
artist_df = pd.read_pickle("../../../data/pickled_dfs/artist_df.pkl")
artist_id_class_count = pd.read_pickle("../../../data/pickled_dfs/larger_spot_artist_id_class_count.pkl")
songs_df = pd.read_pickle("../../../data/pickled_dfs/songs_df.pkl")
song_class_count_df = pd.read_pickle("../../../data/pickled_dfs/song_class_count_df.pkl")
song_class_count_df.rename(columns={'Artist':'Song'}, inplace=True) # Quick clean rename column
class_type_reorg_df = pd.read_pickle("../../../data/pickled_dfs/class_type_reorg_df.pkl")
new_class_types_with_artists = pd.read_pickle("../../../data/pickled_dfs/new_class_types_with_artists.pkl")
master_song_df = pd.read_pickle("../../../data/pickled_dfs/master_song_df.pkl")
# artist_id_class_count has 186 rows with 0LyfQWJT6nXafLPZqxe9Of - it means Various Artists
# and 40 errors

In [4]:
base_clean_df.head()
# Difference from df:
# Dropped null 'instructorBio' rows - only 9
# Dropped rows with null classSongs/classArtists
# Dropped row value with classSongs as float (1 row)
# Populate rows with classType Cycling and classEquipment None to classEquipment = Bike

,classId,className,classDescription,classDifficulty,classDuration,classType,classLength,classLocation,classOriginalAirdate,classRating,classRatingCount,instructorName,instructorBio,classEquipment,classSongs,classArtists,classUrl
0,7f66378211c9476b9b5619bf989f91d0,20 min Peace Meditation,A guided meditation that focuses on cultivatin...,4.3333,20,Meditation,23,psny-studio-2,2021-09-03 13:25:00,0.9847,131,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...",Yoga Block,Meditation 22,RIOPY,https://members.onepeloton.com/classes/bootcam...
1,54ac61803b364b2fa8378acd9f593cdb,15 min Bodyweight Strength,"No equipment, no problem. Join us for a high-e...",5.7755,15,Strength,19,psny-studio-3,2021-09-03 13:19:00,0.9933,297,Olivia Amato,"Born and raised in New York, Olivia grew up pl...",Workout Mat,"California Gurls,Let's Get Loud,Let It Rock (f...","Katy Perry,Snoop Dogg,Jennifer Lopez,Kevin Rud...",https://members.onepeloton.com/classes/bootcam...
2,c75fd4831573483c9d45739aae11d083,20 min Focus Flow: Lower Body,This yoga flow class focuses on poses that eng...,4.3664,20,Yoga,23,psny-studio-2,2021-09-03 12:51:00,1.0000,159,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...","Yoga Blanket,Yoga Block,Yoga Mat","Interlude No 1,Oceansize,She Just Likes To Fig...","James Vincent McMorrow,Oh Wonder,Four Tet,Grim...",https://members.onepeloton.com/classes/bootcam...
3,470086936f7a4723ab5a53cb80b571ff,45 min Pop Bootcamp,Split your workout 50/50 between cardio on the...,7.8312,45,Tread Bootcamp,50,psny-studio-4,2021-09-03 11:56:00,0.9737,152,Olivia Amato,"Born and raised in New York, Olivia grew up pl...","Workout Mat,Medium Weights",34+35 (Remix) (feat. Doja Cat & Megan Thee Sta...,"Ariana Grande,Doja Cat,Megan Thee Stallion,Jus...",https://members.onepeloton.com/classes/bootcam...
4,9680a817bf2149d2b91990c87166a400,20 min Pop Ride,We dare you not to dance as you ride to all th...,7.4000,20,Cycling,24,uk,2021-09-03 07:52:00,1.0000,82,Sam Yo,Sam is a pro at many things but shines when it...,Bike,"Señorita,Marry You,Irreplaceable,What Do You M...","Justin Timberlake,Bruno Mars,Beyoncé,Justin Bi...",https://members.onepeloton.com/classes/bootcam...


In [29]:
len(base_clean_df[base_clean_df['classType'] == 'Cycling'])

7410

In [5]:
base_clean_df['classType'].value_counts(normalize=True)

Cycling           0.442441
Yoga              0.138345
Running           0.118342
Strength          0.117626
Meditation        0.069561
Tread Bootcamp    0.040900
Stretching        0.033974
Walking           0.024719
Cardio            0.010628
Bike Bootcamp     0.003463
Name: classType, dtype: float64

In [6]:
new_class_types_with_artists

,classId_x,instructorName,classType,Cardio,Strength,Stretching,Meditation_x,classId_y,classArtists,Charles Lloyd,Marshall Jefferson,Electric Six,Bjonr,Gerald Alston,Blue Cheer,Madden,Jessy,Corciolli,Party Pupils,Taj Mahal,The Fratellis,The Black Keys,Alina Baraz,Mt. Eden,Álvaro Faria,The Thrills,DJ Jazzy Jeff & The Fresh Prince,Donna Lewis,Culture Beat,The Good Men,Christine Allado,Kinky,Heaven 17,VIP Remix,Aaron Neville,ATFC,Sarah Green,P Money & Riko Dan,Leon Bridges,Blackburner,Lux & Marcusson,MERCE,Luca Schreiner,Tiffany,Sam Smith,Lsd,Peter Johansson,Philharmonic Wind Orchestra,Yelle,Air,Brandon Lay,Ape Drums,CASSIMM,Igor Blaska,Ninja,Baby Boy Da Prince,Round Table Knights,Shaboozey,Marsha Ambrosius,Kai,Tubular Bells,Johnny Nash,Bryce Ryness,J.A.C.K.,SHY FX,Skepta,Tanto Metro & Devonte,Caroline Smith,IZECOLD feat. Molly Ann,Marcy Playground,Ximena Sarinana,Patoranking,Donna Summer,Ben Platt,Jai Wolf,Adrian Delgado,Leon Bolier,Ultraworld,Archie Shepp,Phantom Planet,Groovefinder,Liquid Bloom,Keyone Starr,JONES,Clément Leroux,Still Woozy,Back to Basics,DJ Gollum,Frank Moody,Piano Love Songs,[dunkelbunt],MSTRKRFT,Anna Nalick,Ann Peebles,Outsiders,Switchfoot,Toby Green,Carlos Gallardo,Slow Meadow,The Wanted,Stevie Neale,Raphael Saadiq,Mike Posner,Janine Fagan,Overseer,Tarn Travers,HAIM,Divinyls,Pink,Dmitry Ko,Knuckle G,Simian Mobile Disco,George Kwali,POORSTACY,DJ Snake,die ärzte,Slim Jxmmi,Rebecca E. Covington,Alex Metric,Jack Black,Creed,Caroline Polachek,2 Pistols,Dan Harkna,Alex Somers,Tomcraft,Boss Axis,Krishna Kaur,Kings of Leon,Barbara Mandrell,Indian Wells,Sech,Col3trane,Amanda Wilson,Maduk,Running Touch,Cassandra,Mýa,Miami Rockets,Matrix,Los Gigantes,Morgane Stapleton,Travis Scott,John Gibbons,Quiet,MEUTE,Cello Magic,The Kinks,"Damian ""JR GONG"" Marley",Barnes Courtney,Charles Perry,Caesars,Kelly Osbourne,Lauren Alaina,Just Jack,RAYE,Dick Dale,Ben L'Oncle Soul,The Monkees,Original Mix,Cassiano,Alex Newell,Firebeatz Remix,New Order,Poemme,Joan Osborne,Jan & Dean,Aly-Us,The Barden Bellas,Phaeleh,BROHUG,Richard Judge,EMF,Lang Lang,Vanessa Williams,Social House,Fare Soldi,Roberto Surace,Friendly Fires,Stacie Orrico,Jeezy,Mystery Jets,Lincoln Center Jazz Orchestra,Entheogenic,Shannon,1 Mile North,Junior J,Cali Y El Dandee,Janelle Monáe,24Heavy,Pinguin Mofex,Bowling For Soup,Gregor Tresher,Krystal,The Streets,Leo The Lion,Michael Lange,Utp,Dexys Midnight Runners,Don Trip,Josef Salvat,S1mba,Addal,Astyn Turr,Noah Schnacky,KAMILLE,The Family,My Morning Jacket,Collective Soul,Iamsu!,Astrud Gilberto,Lauren Bennett,Conrank,P.E.O,Sonny Fodera,Est 1987,Kangaroo,Rhye,Cozi,Ria Hall,Miami Sound Machine,George Maple,Christopher Williams,Curtis McClain,Kendrick Lamar,Kut Klose,Sonic Youth,Havana Brown,Official Song UEFA EURO 2016,Bill Withers,Ivan Gough,'Hair' Tribe,J.B,The Who,Joe Jackson,Jamie T,Igi Remix,The Fog,Steven Curtis Chapman,Selena Gomez,Mc Pryme,London Festival Chorus,JBM,Tantric,Danny Verde,Faith No More,The J. Geils Band,Jellyroll,Edson Cordeiro,A$AP Rocky,...,Rob Ruha,Duck Sauce,Kool & The Gang,Luna Aura,David Guetta - Usher,Big & Rich,Little Walter,Veronica,Golden Retriever,Tauren Wells,Ana Bárbara,BLU J,The Endorphins,Nate Ruess,Guru's Jazzmatazz,Bonsai Mammal,Vizin,Will Brennan,Catfish and the Bottlemen,Ambrosian Singers,Tones And I,Baird Hersey & Prana,The Dreem Teem,Lil Mosey,KUČKA,RU.BE,Everything But The Girl,Goetz,Orlando Pops Orchestra,Tigerstyle,Beacon,NOTD,The Blaze,Lalo Ebratt,East Freaks,Charlie Lane,Tall Heights,Jennifer Paige,Big Scoob,Kelsey Lu,Sheryl Lee Ralph,Lucky Charmes,Big Mack,Ice Cube,The Oh Hellos,Reeve Carney,Dannic,Sleepy Bear,Runaway June,Julia Michaels,C2C,Gesaffelstein,Uhuru,Hayley Williams,DARKO,SONS OF HAWAII,Paul Bley,Eddy Duchin,Kaprila Keishing,Emily Warren,Through the Noise,SHAUN,Chris Marshall,Gerardo,The Stooges,Bonzai,the bird and the bee,The Corrs,Chris Marina,Futuristic Polar Bears,Theory of a Deadman,Donna Fargo,Adam Clough,Milosh,Madeon,Steve Martin,Daijo,Less Stress Music Acade

In [25]:
cardio_classes = new_class_types_with_artists[new_class_types_with_artists['Cardio'] == 1]

In [26]:
cardio_classes['classType'].value_counts()

Cycling           7410
Running           1982
Tread Bootcamp     685
Walking            414
Cardio             178
Bike Bootcamp       58
Name: classType, dtype: int64

In [17]:
new_class_types_with_artists.groupby(by=['Cardio', 'Strength', 'Stretching', 'Meditation_x']).sum()

Charles Lloyd  Marshall Jefferson  Electric Six  Bjonr  Gerald Alston  Blue Cheer  Madden  Jessy  Corciolli  Party Pupils  Taj Mahal  The Fratellis  The Black Keys  Alina Baraz  Mt. Eden  Álvaro Faria  The Thrills  DJ Jazzy Jeff & The Fresh Prince  Donna Lewis  Culture Beat  The Good Men  Christine Allado  Kinky  Heaven 17  VIP Remix  Aaron Neville  ATFC  Sarah Green   P Money & Riko Dan  Leon Bridges  Blackburner  Lux & Marcusson  MERCE  Luca Schreiner  Tiffany  Sam Smith  Lsd  Peter Johansson  Philharmonic Wind Orchestra  Yelle  Air  Brandon Lay  Ape Drums  CASSIMM  Igor Blaska  Ninja  Baby Boy Da Prince  Round Table Knights  Shaboozey  Marsha Ambrosius  Kai  Tubular Bells  Johnny Nash  Bryce Ryness  J.A.C.K.  SHY FX  Skepta  Tanto Metro & Devonte  Caroline Smith  IZECOLD feat. Molly Ann  Marcy Playground  Ximena Sarinana  Patoranking  Donna Summer  Ben Platt  Jai Wolf  Adrian Delgado  Leon Bolier  Ultraworld  Archie Shepp  Phantom Planet  \
Cardio Strength Stretching Meditation_x                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
0      0        0          1                         1                   0             0      0              0           0       0      0          0             0          0              0               0            0         0             0            0                                 0            0             0             0                 0      0          0          0              0     0            0                    0             0            0                0      0               0        0          0    0                0                            0      0    0            0          0        0            0      0                   0                    0          0                 0    0              0            0             0         0       0       0                      0               0                        0                 0                0            0             0          0         0               0            0           0             0               0   
                1          0                         0                   0             0      0              0           0       0      0          0             1          1              0               0            0         0             0            0                                 0            0             0             0                 0      0          0          0              0     0            0                    0             1            0                0      0               0        0          2    1                0                            0      0    1            0          0        0            0      0                   0                    0          0                 0    0              0            0             0         0       0       0                      0               0                        0                 1                0            0             2          1         0               0            0           0             0               0   
       1        0          0                 

In [7]:
master_song_df = master_song_df[['peloton_song_name', 'Class Count', 'song_id', 'spotify_song_name', 'album', 'artist',
                                 'popularity', 'duration_ms', 'explicit', 'release_date', 'danceability',
                                 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
                                 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]

In [8]:
master_song_df[master_song_df['song_id'] != 'None']

,peloton_song_name,Class Count,song_id,spotify_song_name,album,artist,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1,Weekend,2,spotify:track:67UJGroXxC22oxzqO8bozE,NaN,NaN,NaN,25,113107,0,1962-01-01,0.738,0.635,8,-8.349,1,0.0312,0.476000,0.00277,0.2940,0.959,98.779,4
2,Supalonely (feat. Gus Dapperton),1,spotify:track:3GZoWLVbmxcBys6g0DLFLf,Supalonely (feat. Gus Dapperton),Hey u x,BENEE,65,223488,True,2020-11-13,0.862,0.631,7,-4.746,1,0.0515,0.291000,0.000209,0.1230,0.841,128.978,4
3,Play (Artful Dodger Mix - Main Mix Radio),1,spotify:track:3qA0jZKzOpfcpXoc1vDkC0,Play - Artful Dodger Mix - Main Mix Radio,Play,Jennifer Lopez,7,229800,False,2001-03-27,0.836,0.604,1,-10.139,0,0.0772,0.029600,0.00025,0.0152,0.970,127.969,4
4,Light Your Ass On Fire (Club Mix) (feat. Pharr...,1,spotify:track:3YhfOgIyJJA2hBgTsHi9WK,Light Your Ass On Fire (feat. Pharrell) - Club...,The Neptunes Present... Clones,The Neptunes,41,219680,True,2003-08-19,0.799,0.705,6,-11.768,0,0.2940,0.030500,4.23e-06,0.3650,0.705,104.878,4
5,Moon and Star,1,spotify:track:6xyanCylUYNcPJp1ao0PSb,Moon and Stars,Moon and Stars,1nonly,56,151587,False,2020-07-20,0.718,0.829,8,-8.487,0,0.1210,0.243000,0,0.1380,0.673,140.003,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29897,What I Got (Reprise),1,spotify:track:4U81vDKgj0QyxtcVl8IKCu,What I Got - Reprise,Sublime,Sublime,46,181600,True,1996-07-30,0.611,0.607,7,-6.514,1,0.0394,0.044200,4.67e-06,0.2450,0.551,92.552,4
29898,White Wedding (Pt. 1 / Remastered 2002),1,spotify:track:0q07lqzKfGJqje97DnEprR,White Wedding - Pt. 1 / Remastered 2002,Greatest Hits,Billy Idol,51,252400,False,2001-01-01,0.656,0.767,9,-7.562,1,0.0375,0.018500,0.00165,0.3710,0.747,147.347,4
29900,Modern Jesus (Charli XCX Remix),2,spotify:track:7yHrK98jg7B5aFKD4nZblp,Modern Jesus - Charli XCX Remix,Evil Friends Remix EP,Portugal. The Man,17,249332,False,2014-08-12,0.295,0.828,1,-4.924,1,0.1600,0.006910,8.09e-05,0.1130,0.516,171.984,4
29901,Sandstorm - Komytea Radio Remix,1,spotify:track:2uUBl4hNsxmHpyR57J8fJw,Sandstorm - Komytea Radio Remix,Sandstorm (The Remixes),Darude,17,234040,False,2007-04-18,0.585,0.970,11,-7.346,0,0.0362,0.000102,0.929,0.1270,0.324,129.010,4


In [9]:
artist_id_class_count.info()
# 82% have additional info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10941 entries, 0 to 10940
Data columns (total 6 columns):
name           10941 non-null object
Class Count    10941 non-null int64
id             10599 non-null object
followers      9017 non-null float64
genres         9017 non-null object
popularity     9017 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 513.0+ KB


In [10]:
artist_id_class_count[~artist_id_class_count['id'].isnull()]

,name,Class Count,id,followers,genres,popularity
38,"""Cats"" 1983 Broadway Cast",1,2cez574BwRD00PGEbNl4SH,3239.0,[],41.0
39,"""Fiddler On The Roof” Motion Picture Orchestra",1,0yultMio5TntZCl8JRRibP,67.0,[],42.0
41,"""The Bodyguard the Musical"" World Premiere Cast",1,13EQcYmeMq9pQEg8VtsALF,874.0,[],9.0
44,'Come From Away' Company,2,3yVB4N2PhQy60dAxzCoNdQ,3272.0,"['broadway', 'show tunes']",49.0
45,'Hair' Tribe,1,3n3yTD6TIoLrzhuR6Wh7kd,203.0,[],34.0
...,...,...,...,...,...,...
10936,yutaka hirasaka,1,0stmdx2IonUUUIlWQ9bLYZ,12363.0,"['j-ambient', 'japanese guitar', 'japanese jaz...",52.0
10937,Álvaro Faria,4,3Vop0LjMsp1743VPBhYJXl,NaN,NaN,NaN
10938,Ásgeir,12,7xUZ4069zcyBM4Bn10NQ1c,252028.0,"['icelandic folk', 'icelandic indie', 'iceland...",60.0
10939,Ådå,1,0Ll3Su2buPphrFk5OdPEvp,1176.0,[],32.0


Note:
classId = 8840f82cdc624aa28a5d1babc51a3916 is the one you need to watch out for (has float classSong)

In [11]:
# Check lengths of dfs
print(f'artist_df: {len(artist_df)}')
print(f'artist_id_class_count: {len(artist_id_class_count)}')
print(f'songs_df: {len(songs_df)}')
print(f'master_song_df: {len(master_song_df)}')
print(f'class_type_reorg_df: {len(class_type_reorg_df)}')
print(f'new_class_types_with_artists: {len(new_class_types_with_artists)}')
# No inconsistencies where there shouldn't be

artist_df: 16748
artist_id_class_count: 10941
songs_df: 16748
master_song_df: 29904
class_type_reorg_df: 16748
new_class_types_with_artists: 16748


#### Notes:
- artist_df has a row for each of the 16748 classes with the lists of artists in that class and 10943 columns
    - column for classId, classArtists (from base_clean_df)
    - then each of the 10941 artists

- artist_id_class_count has 10941 rows for each of the artists with their class count, followers, genres, popularity
    - keep in mind nulls not cleaned yet...
    - keep them in there until you decide what you're doing with the data
    - can clean after merging with final data set


#### Playing with Class Info

In [12]:
class_df = base_clean_df['className'].to_frame()

In [13]:
class_df['className'][0].split('min ')[1]
# Split this off into new column and see if you can parse themes

'Peace Meditation'

# ^Split this off into new column and see if you can parse themes

### Spotify API Work

In [8]:
# Spotify API set up
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
client_id = '4aac5b215c8a4fc591b506b71af7ebf5'
client_secret = '31d999b609424f569382b047ba828b00'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#### Notes:
- Try marrying artist info to base_clean_df first
- When you join, you may need to apply same cleansing steps to the column in base_clean_df for matching

Joining notes
- Final df should have a line for each artist. This means the class data would be duplicated...
- Could you join artist id with classes, then if 

- artist_df joined with base_clean_df would give you artists in class
    - how could this be used to join artist level data
    - add artist columns to joined df (artist_df and base_clean_df), then if artist column = 1 fill in the columns with the artist information?
    - BUT there will be multiple artists

In [9]:
artist_df
# this 

,classId,classArtists,Charles Lloyd,Marshall Jefferson,Electric Six,Bjonr,Gerald Alston,Blue Cheer,Madden,Jessy,...,Maxence Cyrin,B.B. Jay,Yizzy,Bellman,Lauren Pritchard,The Japanese Popstars,James Jacob,Radio Remix,Mayer Hawthorne,John Ryan
0,7f66378211c9476b9b5619bf989f91d0,[RIOPY],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,54ac61803b364b2fa8378acd9f593cdb,"[Katy Perry, Snoop Dogg, Jennifer Lopez, Kevin...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,c75fd4831573483c9d45739aae11d083,"[James Vincent McMorrow, Oh Wonder, Four Tet, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,470086936f7a4723ab5a53cb80b571ff,"[Ariana Grande, Doja Cat, Megan Thee Stallion,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9680a817bf2149d2b91990c87166a400,"[Justin Timberlake, Bruno Mars, Beyoncé, Justi...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16743,a144fcf1595740caa610f6484deb3cdb,"[Guts, Leron Thomas, Soopasoul, FDEL, Basement...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16744,bd89a56c884b48e59ce9c43ab4f4d86f,"[Next, Boyz II Men, Blues Traveler, Eagle-Eye ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16745,e5a99c83d7fc4f0686c68b02e5932204,"[Daphne Willis, Gizzle, Lo Air, Sigala, Ella E...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16746,c57e123004d04df1bfae94c5e4399bb3,"[Robyn, No Doubt, C & C Music Factory, Technot...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
artist_id_class_count

,name,Class Count,id,followers,genres,popularity
0,ACE HOOD,1,NaN,NaN,NaN,NaN
1,AKON,1,NaN,NaN,NaN,NaN
2,Cambridge,1,NaN,NaN,NaN,NaN
3,Cr3on,6,NaN,NaN,NaN,NaN
4,DJ Deeon,13,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
10936,yutaka hirasaka,1,0stmdx2IonUUUIlWQ9bLYZ,12363.0,"['j-ambient', 'japanese guitar', 'japanese jaz...",52.0
10937,Álvaro Faria,4,3Vop0LjMsp1743VPBhYJXl,NaN,NaN,NaN
10938,Ásgeir,12,7xUZ4069zcyBM4Bn10NQ1c,252028.0,"['icelandic folk', 'icelandic indie', 'iceland...",60.0
10939,Ådå,1,0Ll3Su2buPphrFk5OdPEvp,1176.0,[],32.0
